In [ ]:
import pandas as pd
import dotenv
import os
from libraries.client_stashapp import get_stashapp_client
from libraries.StashDbClient import StashDbClient

dotenv.load_dotenv()

stash = get_stashapp_client()

stashbox_client = StashDbClient(
    os.getenv("STASHDB_ENDPOINT"),
    os.getenv("STASHDB_API_KEY"),
)

In [82]:
galleries = stash.find_galleries(fragment="id title date files { basename } studio { id name }")
df_galleries = pd.DataFrame(galleries)

# Filter galleries with a single file
df_single_file_galleries = df_galleries[df_galleries['files'].apply(lambda x: len(x) == 1)].copy()

# Extract basename of the single file
df_single_file_galleries['file_basename'] = df_single_file_galleries['files'].apply(lambda x: os.path.basename(x[0]['basename']))
df_single_file_galleries['studio_id'] = df_single_file_galleries['studio'].apply(lambda x: x['id'])
df_single_file_galleries['studio_name'] = df_single_file_galleries['studio'].apply(lambda x: x['name'])

# Parse date from file_basename


In [48]:
# Select only the specified columns
df_single_file_galleries_dates = df_single_file_galleries[['id', 'date', 'title', 'file_basename']].copy()

# Parse date from file_basename and create a new column
df_single_file_galleries_dates.loc[:, 'parsed_date'] = df_single_file_galleries_dates['file_basename'].str.extract(r'(\d{4}-\d{2}-\d{2})')[0]

# Filter rows where date is null, parsed_date is not null, and parsed_date is not "0001-01-01"
df_single_file_galleries_dates = df_single_file_galleries_dates[
    (df_single_file_galleries_dates['date'].isna()) & 
    (df_single_file_galleries_dates['parsed_date'].notna()) & 
    (df_single_file_galleries_dates['parsed_date'] != "0001-01-01")
]


In [49]:
for index, row in df_single_file_galleries_dates.iterrows():
    stash.update_gallery({
        "id": row["id"],
        "date": row["parsed_date"]
    })

In [16]:
# Parse studio code from file_basename

df_single_file_galleries_studio_codes = df_single_file_galleries.copy()
df_single_file_galleries_studio_codes["studio_name"] = df_single_file_galleries_studio_codes["studio"].apply(lambda x: x["name"])
df_single_file_galleries_studio_codes["studio_code"] = df_single_file_galleries_studio_codes["file_basename"].str.split("-").str[4].str.strip()

selected_studios = ["WowGirls", "WowPorn", "All Fine Girls", "Ultra Films"]
df_selected_studios_galleries = df_single_file_galleries_studio_codes[df_single_file_galleries_studio_codes["studio_name"].isin(selected_studios)]
df_selected_studios_galleries = df_selected_studios_galleries[df_selected_studios_galleries["studio_code"].notna()]


In [17]:
for index, row in df_selected_studios_galleries.iterrows():
    stash.update_gallery({
        "id": row["id"],
        "code": row["studio_code"]
    })


# Parse title from file_basename

In [83]:
df_single_file_galleries_titles = df_single_file_galleries.copy()
df_single_file_galleries_titles = df_single_file_galleries_titles[df_single_file_galleries_titles["title"] == ""]
df_single_file_galleries_titles["parsed_title"] = df_single_file_galleries_titles['file_basename'].apply(lambda x: 
    ' - '.join(x.split(' - ')[3:-1]).strip()
)

In [85]:
df_selected_galleries_titles = df_single_file_galleries_titles.copy()
selected_studios = ["Blacked", "Blacked Raw", "Deeper", "Milfy", "Slayed", "Tushy", "Tushy Raw", "Vixen"]
df_selected_galleries_titles = df_selected_galleries_titles[df_selected_galleries_titles["studio_name"].isin(selected_studios)]
df_selected_galleries_titles = df_selected_galleries_titles[df_selected_galleries_titles["parsed_title"].notna()]

In [68]:
for index, row in df_selected_galleries_titles.iterrows():
    stash.update_gallery({
        "id": row["id"],
        "title": row["parsed_title"]
    })

# Parse studio from file_basename


In [ ]:
# Select only the specified columns and create a copy
df_single_file_galleries_studios = df_single_file_galleries[['id', 'studio', 'title', 'file_basename']].copy()

In [ ]:
selected_studio = "WowPorn"

def clean_data(df_single_file_galleries_studios):
    # Derive column 'gallery_date' from column: 'file_basename'
    # Transform based on the following examples:
    #    file_basename                                             Output
    # 1: "MetArt - 2010-02-08 - 20100208WHY_METART_______ - Why => "2010-02-08"
    #    Metart... -) - Ariel Piper Fawn [high].zip"
    df_single_file_galleries_studios.insert(4, "gallery_date", df_single_file_galleries_studios["file_basename"].str.split(" ").str[2])

    # Derive column 'gallery_studio' from column: 'file_basename'
    # Transform based on the following examples:
    #    file_basename                                                 Output
    # 1: "MetArt - 2008-09-22 - 20080922STEAMING - Steaming - Ariel => "MetArt"
    #    Piper Fawn [high].zip"
    df_single_file_galleries_studios.insert(4, "gallery_studio", df_single_file_galleries_studios["file_basename"].str.split("-").str[0].str.strip())

    # Derive column 'gallery_title' from column: 'file_basename'
    # Transform based on the following examples:
    #    file_basename                                                Output
    # 1: "MetArt - 2022-05-25 - 20220525PRESENTING_KELLY_COLLINS - => "Presenting Kelly Collins"
    #    Presenting Kelly Collins - Kelly Collins [high].zip"
    df_single_file_galleries_studios.insert(4, "gallery_title", df_single_file_galleries_studios["file_basename"].str.split("-").str[5].str.strip())

    # Filter rows based on columns: 'gallery_date', 'gallery_title', 'gallery_studio'
    df_single_file_galleries_studios = df_single_file_galleries_studios[(df_single_file_galleries_studios['gallery_date'].notna()) & (df_single_file_galleries_studios['gallery_title'].notna()) & (df_single_file_galleries_studios['gallery_studio'].notna())]

    # Filter rows based on column: 'gallery_studio'
    df_single_file_galleries_studios = df_single_file_galleries_studios[df_single_file_galleries_studios['gallery_studio'] == selected_studio]

    # Derive column 'gallery_performers' from column: 'file_basename'
    # Transform based on the following examples:
    #    file_basename                                                Output
    # 1: "MetArt - 2022-02-24 - 20220224GRATIFY - Gratify - Stella => "Stella Cardo"
    #    Cardo [high].zip"
    # 2: "MetArt - 2011-11-27 - 20111127PRESENTING_MICHELLE -      => "Michelle H"
    #    Presenting Michelle - Michelle H [high].zip"
    # 3: "MetArt - 2010-02-08 - 20100208WHY_METART_______ - Why    => "Ariel Piper Fawn"
    #    Metart... -) - Ariel Piper Fawn [high].zip"
    df_single_file_galleries_studios.insert(4, "gallery_performers", df_single_file_galleries_studios.apply(lambda row : row["file_basename"][row["file_basename"].rfind("-") + 2:row["file_basename"].rfind(" ")], axis=1))
    
    # Derive column 'gallery_performers_separated' from column: 'gallery_performers'
    def gallery_performers_separated(gallery_performers):
        import re

        """
        Transform based on the following examples:
           gallery_performers                Output
        1: "Ariel Piper Fawn"             => "Ariel Piper Fawn"
        2: "Melisa A & Caprice A"         => "Melisa A, Caprice A"
        3: "Michelle H"                   => "Michelle H"
        4: "Vera"                         => "Vera"
        5: "Vera & Michelle H"            => "Vera, Michelle H"
        6: "Vera, Michelle H & Ariel Piper Fawn" => "Vera, Michelle H, Ariel Piper Fawn"
        """
        # Split by '&' and ',' to handle both separators
        performers = [p.strip() for p in re.split('[&,]', gallery_performers)]
        
        # Remove any empty strings that might result from splitting
        performers = [p for p in performers if p]
        
        # Join the performers with a comma and space
        return ', '.join(performers)

    df_single_file_galleries_studios.insert(5, "gallery_performers_separated", df_single_file_galleries_studios.apply(lambda row : gallery_performers_separated(row["gallery_performers"]), axis=1))

    df_single_file_galleries_studios.insert(4, "gallery_studio_code", df_single_file_galleries_studios["file_basename"].str.split("-").str[4].str.strip())

    return df_single_file_galleries_studios

df_single_file_galleries_studios_clean = clean_data(df_single_file_galleries_studios.copy())
df_single_file_galleries_studios_clean.head()

In [ ]:
# There might be multiple performers in gallery_performers_separated, so we need to split them into a list and create a new data frame of just the performers so we can match those to Stash performers in df_performers
df_performers_list = df_single_file_galleries_studios_clean['gallery_performers_separated'].str.split(',').explode().str.strip().unique().tolist()

In [ ]:
studios = stash.find_studios(fragment="id name")
df_studios = pd.DataFrame(studios)

In [ ]:
performers = stash.find_performers(fragment="id name alias_list")
df_performers = pd.DataFrame(performers)

In [ ]:
# Create a DataFrame with all performers from df_performers_list
df_performers_with_stash_ids = pd.DataFrame({'name': df_performers_list})

# Function to check if a name matches any alias in the alias_list
def match_alias(name, row):
    if pd.isna(row['alias_list']).all():
        return False
    return any(name.lower() == alias.lower() for alias in row['alias_list'] if pd.notna(alias) and alias is not None)

# Merge with df_performers to get stash_ids where available, first by name
df_performers_with_stash_ids = df_performers_with_stash_ids.merge(
    df_performers[['id', 'name', 'alias_list']], 
    on='name', 
    how='left'
)

# For unmatched performers, try matching by alias
for idx, row in df_performers_with_stash_ids[df_performers_with_stash_ids['id'].isna()].iterrows():
    match = df_performers[df_performers.apply(lambda x: match_alias(row['name'], x), axis=1)]
    if not match.empty:
        df_performers_with_stash_ids.loc[idx, 'id'] = match.iloc[0]['id']

# Rename 'id' column to 'stash_id'
df_performers_with_stash_ids = df_performers_with_stash_ids.rename(columns={'id': 'stash_id'})

# Drop the 'alias_list' column as it's no longer needed
df_performers_with_stash_ids = df_performers_with_stash_ids.drop(columns=['alias_list'])

# Sort the DataFrame by name for better readability
df_performers_with_stash_ids = df_performers_with_stash_ids.sort_values('name').reset_index(drop=True)

In [ ]:
# Match performers in df_performers_with_stash_ids to df_single_file_galleries_studios_clean and create a new column with the stash_ids
def get_performer_ids(performers):
    performer_list = performers.split(',')
    ids = []
    for performer in performer_list:
        performer = performer.strip()
        matched_ids = df_performers_with_stash_ids[
            (df_performers_with_stash_ids['name'].str.contains(performer, case=False, na=False)) &
            (df_performers_with_stash_ids['stash_id'].notna())
        ]['stash_id'].tolist()
        ids.extend(matched_ids)
    return ids if ids else None

df_single_file_galleries_studios_clean['stash_ids'] = df_single_file_galleries_studios_clean['gallery_performers_separated'].apply(get_performer_ids)


In [ ]:
def get_studio_id(studio_name):
    matched_ids = df_studios[
        (df_studios['name'].str.contains(studio_name, case=False, na=False)) &
        (df_studios['id'].notna())
    ]['id'].tolist()
    return matched_ids[0] if matched_ids else None

df_single_file_galleries_studios_clean['studio_id'] = df_single_file_galleries_studios_clean['gallery_studio'].apply(get_studio_id)

In [ ]:
# Manually override the studio ID
df_single_file_galleries_studios_clean['studio_id'] = '103'


In [ ]:
first_gallery = df_single_file_galleries_studios_clean.iloc[0]
print(first_gallery)

In [ ]:
stash.update_gallery({
    "id": first_gallery["id"],
    "title": first_gallery["gallery_title"],
    "date": first_gallery["gallery_date"],
    "studio_id": first_gallery["studio_id"],
    "performer_ids": first_gallery["stash_ids"],
    "code": first_gallery["gallery_studio_code"]
})

In [ ]:
for index, row in df_single_file_galleries_studios_clean.iterrows():
    stash.update_gallery({
        "id": row["id"],
        "title": row["gallery_title"],
        "date": row["gallery_date"],
        "studio_id": row["studio_id"],
        "performer_ids": row["stash_ids"],
        "code": row["gallery_studio_code"]
    })


# Matching galleries and scenes

In [131]:
standalone_gallery_tag = stash.find_tag("Standalone Gallery")

In [155]:
galleries = stash.find_galleries({
    "scenes": { "value": [], "modifier": "IS_NULL" },
    "tags": { "value": [], "modifier": "INCLUDES", "excludes": [standalone_gallery_tag["id"]] }
}, fragment="id title date code studio { id name }")
df_galleries_for_matching = pd.DataFrame(galleries)
df_galleries_for_matching['gallery_id'] = df_galleries_for_matching['id']
df_galleries_for_matching['gallery_date'] = df_galleries_for_matching['date']
df_galleries_for_matching["gallery_title"] = df_galleries_for_matching["title"]
df_galleries_for_matching["gallery_studio_id"] = df_galleries_for_matching["studio"].apply(lambda x: x["id"])
df_galleries_for_matching["gallery_studio_name"] = df_galleries_for_matching["studio"].apply(lambda x: x["name"])
df_galleries_for_matching = df_galleries_for_matching[['gallery_id', 'gallery_date', 'gallery_title', 'gallery_studio_id', 'gallery_studio_name']]

In [156]:
scenes = stash.find_scenes({ "galleries": { "value": [], "modifier": "IS_NULL" }, "studios": { "value": [], "modifier": "NOT_NULL" } }, fragment="id title date code studio { id name }")
df_scenes_for_matching = pd.DataFrame(scenes)
df_scenes_for_matching['scene_id'] = df_scenes_for_matching['id']
df_scenes_for_matching['scene_date'] = df_scenes_for_matching['date']
df_scenes_for_matching["scene_title"] = df_scenes_for_matching["title"]
df_scenes_for_matching["scene_studio_id"] = df_scenes_for_matching["studio"].apply(lambda x: x["id"])
df_scenes_for_matching["scene_studio_name"] = df_scenes_for_matching["studio"].apply(lambda x: x["name"])
df_scenes_for_matching = df_scenes_for_matching[['scene_id', 'scene_date', 'scene_title', 'scene_studio_id', 'scene_studio_name']]

In [157]:
# Merge the data frames by gallery_title and scene_title, and ensure studio IDs match
df_merged = pd.merge(df_galleries_for_matching, df_scenes_for_matching, 
                     left_on=['gallery_title', 'gallery_studio_id'], 
                     right_on=['scene_title', 'scene_studio_id'], 
                     how='inner')

df_selected_merged = df_merged[df_merged["gallery_studio_name"] == "PornFidelity"].copy()
df_selected_merged["date_diff"] = (pd.to_datetime(df_selected_merged["gallery_date"]) - pd.to_datetime(df_selected_merged["scene_date"])).dt.days

In [151]:
for index, row in df_selected_merged.iterrows():
    scene_id = row["scene_id"]
    gallery_id = row["gallery_id"]
    
    stash.update_scene({
        "id": scene_id,
        "gallery_ids": [gallery_id]
    })

In [ ]:
# print first row1
print(df_merged.iloc[0])



In [ ]:
stash.update_gallery({
    "id": df_merged.iloc[0]["gallery_id"],
    "scene_ids": [df_merged.iloc[0]["scene_id"]]
})

In [ ]:
for index, row in df_merged.iterrows():
    stash.update_gallery({
        "id": row["gallery_id"],
        "scene_ids": [row["scene_id"]]
    })

# Get metadata from PostgreSQL database

In [ ]:
%pip install psycopg2-binary
%pip install sqlalchemy

In [ ]:
import os
from sqlalchemy import create_engine
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get the connection string from environment variable
connection_string = os.getenv('CONNECTION_STRING')

# Create SQLAlchemy engine
engine = create_engine(connection_string)

# Example query
query = """
SELECT d.*, r.url, r.description, r.short_name
FROM downloads d
JOIN releases r ON d.release_uuid = r.uuid
JOIN sites s ON r.site_uuid = s.uuid
WHERE s.name = 'MetArt'
"""

# Read data from PostgreSQL into a pandas DataFrame
df_downloads = pd.read_sql_query(query, engine)

# Close the database connection
engine.dispose()


In [ ]:
df_galleries = pd.DataFrame(galleries)

# Filter galleries with a single file
df_single_file_galleries = df_galleries[df_galleries['files'].apply(lambda x: len(x) == 1)].copy()

# Extract basename of the single file
df_single_file_galleries.loc[:, 'file_basename'] = df_single_file_galleries['files'].apply(lambda x: os.path.basename(x[0]['basename']))

# Reset index for cleaner output
df_single_file_galleries = df_single_file_galleries.reset_index(drop=True)


In [ ]:
# Merge df_single_file_galleries with df_downloads based on file_basename and saved_filename
df_merged = pd.merge(
    df_single_file_galleries,
    df_downloads,
    left_on='file_basename',
    right_on='saved_filename',
    how='inner'
)

# Check the number of matches
print(f"Number of matched galleries: {len(df_merged)}")

# Display the first few rows of the merged dataframe
print(df_merged[['file_basename', 'saved_filename', 'created_at']].head())

# Check for galleries that didn't match
unmatched_galleries = df_single_file_galleries[~df_single_file_galleries['file_basename'].isin(df_downloads['saved_filename'])]
print(f"Number of unmatched galleries: {len(unmatched_galleries)}")

# If there are unmatched galleries, you might want to investigate why
if len(unmatched_galleries) > 0:
    print("Sample of unmatched galleries:")
    print(unmatched_galleries['file_basename'].head())


In [ ]:
# Get the first gallery
first_gallery = df_merged.iloc[0]
print(first_gallery)


In [ ]:
for idx, row in df_merged.iterrows():
    stash.update_gallery({
        "id": row["id"],
        "code": row["short_name"],
        "url": row["url"],
        "details": row["description"]
    })

# Get titles from matched scenes

In [120]:
studio_ids = []
studio_names = ["Blacked", "Blacked Raw", "Deeper", "Milfy", "Slayed", "Tushy", "Tushy Raw", "Vixen"]
for studio_name in studio_names:
    studio_candidates = stash.find_studios({ "name": { "value": studio_name, "modifier": "EQUALS" } }, fragment="id name")
    if (len(studio_candidates) == 0):
        print(f"No studio found with name {studio_name}")
    elif (len(studio_candidates) == 1):
        studio_ids.append(studio_candidates[0]["id"])
    else:
        print(f"Found multiple studios with name {studio_name}")

In [122]:
galleries_with_scenes = stash.find_galleries({
    "scenes": { "value": [], "modifier": "NOT_NULL" },
    "studios": { "value": studio_ids, "modifier": "INCLUDES" }
}, fragment="id title date studio { name } performers { id name } scenes { id title date details performers { id name } }")

df_galleries_with_scenes = pd.DataFrame(galleries_with_scenes)

df_galleries_with_scenes["gallery_performer_ids"] = df_galleries_with_scenes["performers"].apply(lambda x: [performer["id"] for performer in x])
df_galleries_with_scenes["gallery_performer_names"] = df_galleries_with_scenes["performers"].apply(lambda x: [performer["name"] for performer in x])

df_galleries_with_scenes["scene_title"] = df_galleries_with_scenes["scenes"].apply(lambda x: x[0]["title"] if x else None)
df_galleries_with_scenes["scene_details"] = df_galleries_with_scenes["scenes"].apply(lambda x: x[0]["details"] if x else None)
df_galleries_with_scenes["scene_performer_ids"] = df_galleries_with_scenes["scenes"].apply(lambda x: [performer["id"] for performer in x[0]["performers"]])
df_galleries_with_scenes["scene_performer_names"] = df_galleries_with_scenes["scenes"].apply(lambda x: [performer["name"] for performer in x[0]["performers"]])

In [123]:
for index, row in df_galleries_with_scenes.iterrows():
    stash.update_gallery({
        "id": row["id"],
        "details": row["scene_details"],
        "performer_ids": row["scene_performer_ids"],
        "title": row["scene_title"]
    })